## Classification des poches protéiques en fonction du type de druggabilité, par un CNN

### 1) Préparation des données

In [1]:
import keras
import numpy as np
from random import shuffle

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from keras.layers import Dense, Flatten, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.callbacks.callbacks import EarlyStopping, Callback, ModelCheckpoint
from keras.layers import add, Activation
from keras.layers import Conv3D, MaxPool3D
from keras.models import Sequential, load_model
from os import listdir

Using TensorFlow backend.


In [2]:
PATH_TRAIN = "../data/x_train"
PATH_TEST = "../data/x_test"

In [3]:
def load_x(path, control):
    X = [np.load("{}/{}".format(path, pocket))
         for pocket in listdir(path)]
    X = [np.squeeze(array) for array in X]
    X = np.array(X)
    X = np.moveaxis(X, 1, -1)
    return X

def load_y(path, nucleotid, heme, steroid, control):
    Y = []
    for pocket in listdir(path):
        if pocket in nucleotid:
            Y.append(1)
        elif pocket in heme:
            Y.append(2)
        elif pocket in steroid:
            Y.append(3)
        elif pocket in control:
            Y.append(0)
    Y  = np.array(Y)
    return Y

def one_hot_encoding(y):
    classes = LabelEncoder()
    integer_encoding = classes.fit_transform(y)
    one_hot_Y = keras.utils.to_categorical(integer_encoding)
    return one_hot_Y

def list_generator(file):
    with open(file, "r") as filin:
        liste = ["{}.npy".format(line[:-1]) for line in filin]
    return liste

In [4]:
#X_train est une liste d'array de dimensions (1,14,32,32,32)
#Il est constitué des 346 premières poches
nucleotid = list_generator("nucleotide.list.txt")
heme = list_generator("heme.list.txt")
steroid = list_generator("steroid.list.txt")
control = list_generator("control.list.txt")

X_train = load_x(PATH_TRAIN, control)
X_test = load_x(PATH_TEST, control)
Y_train = load_y(PATH_TRAIN, nucleotid, heme, steroid, control)
Y_test = load_y(PATH_TEST, nucleotid, heme, steroid, control)
one_hot_Y_train = one_hot_encoding(Y_train)
one_hot_Y_test = one_hot_encoding(Y_test)
print(X_train.shape)
print(X_test.shape)
print(one_hot_Y_train.shape)
print(one_hot_Y_test.shape)

(229, 32, 32, 32, 14)
(357, 32, 32, 32, 14)
(229, 4)
(357, 4)


In [28]:
print(len(listdir(PATH_TRAIN)))
print(len(listdir(PATH_TEST)))
print(len(Y_train))
print(len(Y_test))

229
357
229
357


Pour les Y va créer un code pour savoir quelle est la classe de druggabilité de la poche:
<ul>
    <li/> 1 : nucléotide
    <li/> 2 : hème
    <li/> 3 : stéroïde
    <li/> 0 : control

Attention cependant, j'ai pris les 346 première poches, or peut être qu'il y a un déséquilibre de classe: beaucoup de poches dans une classe et beaucoup dans d'autres. On va regarder. 

In [5]:
nt= 0
hem = 0
ste = 0
ctr = 0

for i in range(0,one_hot_Y_train.shape[0]):
    if one_hot_Y_train[i,0]:
        nt += 1
    elif one_hot_Y_train[i,1]:
        hem += 1
    elif one_hot_Y_train[i,2]:
        ste += 1
    else:
        ctr += 1

In [7]:
print(nt)
print(hem)
print(ste)
print(ctr)
print("{}+{}+{}+{} = {}".format(nt,hem,ste,ctr, nt+hem+ste+ctr))
print(len(one_hot_Y_train))

52
68
7
0
52+68+7+0 = 127
127
1946


On s'aperçoit qu'on a très peu de poches druggables par les stéroïdes, c'est une bonne chose car ce sont des faux positifs. Les stéroïdes jouent le rôle de contrôle négatif. A part ça, pas de prédominance particulière d'une classe par rapport à l'autre.

### 2) Construction du modèle

In [23]:
def model_one():
    model = Sequential()
    model.add(Conv3D(filters = 32, kernel_size = 14, strides=1, padding= "same", activation = "relu", kernel_initializer="he_normal"))
    model.add(Conv3D(filters=22, kernel_size = 7, strides=1, padding= "same", activation = "relu", kernel_initializer="he_normal"))
    model.add(Dropout(0.4))
    model.add(MaxPool3D(pool_size = 2, strides = 1, padding = "same"))
    model.add(Dropout(0.4))
    model.add(Flatten())
    model.add(Dense(units = 75, activation = "relu"))
    model.add(Dropout(0.4))
    model.add(Dense(units = 4, activation = "softmax"))
    model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=['accuracy'])
    return model

In [8]:
np.random.seed(1000)
critor = EarlyStopping(monitor = "val_loss", patience = 2, mode = "min")
my_model = model_one()

#best_model_path = "../results/my_model"+".h5"
#best_model = ModelCheckpoint(best_model_path, monitor = "val_loss", verbose = 2, save_best_only = True)
#my_best_model = load_model("../results/my_model.h5")

my_model.fit(X_train[0:200,:,:,:,:], one_hot_Y_train[0:200,:], epochs = 5, batch_size = 15,
                  validation_split = 0.1, callbacks = [critor])


Train on 180 samples, validate on 20 samples
Epoch 1/5
180/180 [==============================] - 100s 557ms/step - loss: 39.0845 - accuracy: 0.3111 - val_loss: 1.2894 - val_accuracy: 0.4000
Epoch 2/5
180/180 [==============================] - 100s 553ms/step - loss: 1.3520 - accuracy: 0.2500 - val_loss: 1.3275 - val_accuracy: 0.3000
Epoch 3/5
180/180 [==============================] - 99s 549ms/step - loss: 1.2666 - accuracy: 0.3611 - val_loss: 1.2987 - val_accuracy: 0.3500


### 3) Evaluation du modèle

In [9]:
evaluation = my_model.evaluate(X_test, one_hot_Y_test)
print(evaluation)

357/357 [==============================] - 34s 97ms/step
[1.3030099234327215, 0.1260504275560379]


Avec les paramètres suivants:
<ul>
    <li/> filters = 24
    <li/> kernel_size = 12
    <li/> pool_size = 2
    <li/> units = 50
    
J'obtient une val loss de 1.29 et une accuracy de 0.37

In [ ]:
training = KerasClassifier(build_fn = model_one, epochs = 5, batch_size=15, verbose=0)
kfold = KFold(n_splits = 5, shuffle=True)
cv_result = cross_val_score(training, X_train, one_hot_Y_train, cv = kfold)
print(cv_result)
print("%.2f%%(%2d%%)"%(cv_result.mean()*100, cv_result.std()*100))

In [29]:
predictions = my_model.predict(X_test)

tp = 0
fp = 0
tn = 0
fn = 0

for i in range(predictions.shape[0]):
    maxi = max(predictions[i,:])
    if maxi == predictions[i, 0]:
        classe = 0
    elif maxi == predictions[i,1]:
        classe = 1
    elif maxi == predictions[i,2]:
        classe = 2
    elif maxi == predictions[i,3]:
        classe = 3
        
    if (one_hot_Y_test[i, 1] == 1.0) and (classe == 1):
        tp += 1
    elif (one_hot_Y_test[i, 2] == 1.0) and (classe == 2):
        tp += 1
    elif (one_hot_Y_test[i, 3] == 1.0) and (classe == 3):
        tp += 1
    elif (one_hot_Y_test[i, 0] == 1.0) and (classe == 1):
        fp += 1
    elif (one_hot_Y_test[i, 0] == 1.0) and (classe == 2):
        fp += 1
    elif (one_hot_Y_test[i, 0] == 1.0) and (classe == 3):
        fp += 1
    elif (one_hot_Y_test[i, 0] == 1.0) and (classe == 0):
        tn += 1
    elif (one_hot_Y_test[i, 0] == 0.0) and (classe == 0):
        fn += 1
        
from math import sqrt

print("TP:{:.2f}%".format(tp*100/len(predictions)))
print("FP:{:.2f}%".format(fp*100/len(predictions)))
print("TN:{:.2f}".format(tn*100/len(predictions)))
print("FN:{:.2f}".format(fn*100/len(predictions)))
print("ACC = {:.2f}%".format((tp+tn)*100/(tp+tn+fp+fn)))
print("PPV = {:.2f}%".format(tp*100/(tp+fp)))
print("TNR = {:.2f}%".format(tn*100/(tn+fp)))
print("TPR = {:.2f}%".format(tp*100/(tp+fn)))
print("FPR = {:.2f}%".format(fp*100/(fp+tn)))
print("MCC = {:.2f}".format(((tn*tp)-(fp*fn))/sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))))

TP:12.04%
FP:22.97%
TN:0.56
FN:6.72
ACC = 29.80%
PPV = 34.40%
TNR = 2.38%
TPR = 64.18%
FPR = 97.62%
